In [1]:
# Install needed libraries
!pip install -q transformers sentence-transformers

In [2]:
import os, glob, pandas as pd
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
!ls /content/drive

'My Drive'


In [10]:
!ls "/content/drive/My Drive/FairwAI/EachCity"

Labeled


In [15]:
clean_dir = '/content/drive/My Drive/FairwAI/EachCity/Clean/'
out_dir   = '/content/drive/My Drive/FairwAI/EachCity/Labeled/'
os.makedirs(out_dir, exist_ok=True)

In [17]:
!ls "/content/drive/My Drive/FairwAI/EachCity/Clean/"

ls: cannot access '/content/drive/My Drive/FairwAI/EachCity/Clean/': No such file or directory


In [6]:
# Direct-bias: Toxic-BERT (Jigsaw fine-tuned)
tox_detector = pipeline(
    "text-classification",
    model="unitary/toxic-bert",
    return_all_scores=True
)

# Indirect-bias: Sentence embeddings
embedder = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/811 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
# Comprehensive Seed Lexicon for Bias Detection

seed_terms = [
    # Direct Race/Ethnicity Slurs
    "nigger", "coon", "gook", "chink", "spic", "mick", "dago", "wetback", "kike",
    "paki", "raghead", "camel jockey", "jungle bunny",

    # Direct Religion Slurs
    "kafir", "heathen", "infidel", "heretic", "mzungu", "raghead",

    # Direct Gender/Sexual Orientation Slurs
    "faggot", "dyke", "tranny", "homo", "fairy", "sissy", "queer", "lezbo",
    "slut", "whore", "hoe", "bimbo",

    # Direct Disability Slurs
    "retard", "cripple", "spaz", "gimp", "mong", "psycho", "schizo",
    "lame", "deaf and dumb",

    # Direct Ageism Slurs
    "old hag", "geezer", "boomer", "codger", "old bastard",

    # Socioeconomic/Regional Slurs
    "redneck", "hick", "hillbilly", "trailer trash", "welfare queen", "pinko",
    "white trash",

    # Microaggression & Implicit Bias Phrases
    # Race/Nationality
    "where are you really from", "you speak good english", "you people",
    "colored people", "urban", "ghetto", "inner city",

    # Gender
    "for a woman you", "you’re so articulate", "you’re so emotional",
    "not like other girls", "man up", "grow a spine", "you throw like a girl",

    # Religion
    "bless you", "holy water", "oh lord", "god bless",

    # Disability
    "i’m so ocd", "you’re so brave", "special needs", "wheelchair bound",

    # Ageism
    "over the hill", "get off my lawn", "back in my day", "senile",

    # Professional / Role Stereotypes
    "glorified babysitters", "just a bartender", "man’s job", "women’s work",

    # Threatening / Harassment
    "make you regret", "kill you", "burn", "attack you", "shoot", "beat you",

    # Sexual Harassment / Objectification
    "nice rack", "hot", "lick", "cum", "pussy", "dick", "asshole",

    # Other Derogatory Adjectives
    "trash", "scum", "idiot", "moron", "filthy", "disgusting", "worthless",
]


In [8]:
seed_embs = embedder.encode(seed_terms, convert_to_tensor=True)

In [11]:
# Processing Loop
# -------------------------------
for fp in glob.glob(os.path.join(clean_dir, '*_cleaned.csv')):
    city_key = os.path.basename(fp).replace('_cleaned.csv','')
    df       = pd.read_csv(fp)

    # Ensure text is string
    df['clean_text'] = df['clean_text'].fillna('').astype(str)

    # Toxic-BERT Scoring
    tox_scores = []
    for text in df['clean_text']:
        out = tox_detector(text, truncation=True)[0]
        scores = {r['label']: r['score'] for r in out}
        # pick the max between direct toxicity and identity_hate
        tox_val = max(scores.values())
        tox_scores.append(tox_val)
    df['toxicity_score'] = tox_scores

    # Embedding Similarity
    # batch-encode all reviews
    rev_embs = embedder.encode(df['clean_text'].tolist(), convert_to_tensor=True)
    sims     = util.cos_sim(rev_embs, seed_embs).max(dim=1).values.cpu().numpy()
    df['bias_similarity'] = sims

    # Combined Flag
    # Mark biased if either score exceeds its threshold
    df['is_biased'] = (
        (df['toxicity_score'] >= 0.5) |
        (df['bias_similarity'] >= 0.5)
    )

    # Save Labeled CSV
    out_fp = os.path.join(out_dir, f'{city_key}_labeled.csv')
    df.to_csv(out_fp, index=False)
    print(f"{city_key}: saved {len(df)} rows to {out_fp}")

In [12]:
for fp in glob.glob(os.path.join(clean_dir, '*_cleaned.csv')):
    city_key = os.path.basename(fp).replace('_cleaned.csv','')
    print(city_key)